In [2]:
# load data

import scipy.io

mat = scipy.io.loadmat('Xtrain.mat')

mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Apr 29 11:11:31 2019',
 '__version__': '1.0',
 '__globals__': [],
 'Xtrain': array([[ 86],
        [141],
        [ 95],
        [ 41],
        [ 22],
        [ 21],
        [ 32],
        [ 72],
        [138],
        [111],
        [ 48],
        [ 23],
        [ 19],
        [ 27],
        [ 59],
        [129],
        [129],
        [ 58],
        [ 27],
        [ 19],
        [ 24],
        [ 46],
        [112],
        [144],
        [ 73],
        [ 30],
        [ 20],
        [ 19],
        [ 37],
        [ 92],
        [152],
        [ 93],
        [ 36],
        [ 20],
        [ 18],
        [ 29],
        [ 71],
        [146],
        [117],
        [ 46],
        [ 23],
        [ 18],
        [ 22],
        [ 52],
        [128],
        [142],
        [ 62],
        [ 26],
        [ 17],
        [ 19],
        [ 37],
        [100],
        [158],
        [ 86],
        [ 32],
        [ 17],
        [ 1

In [4]:
data = mat['Xtrain']
data

array([[ 86],
       [141],
       [ 95],
       [ 41],
       [ 22],
       [ 21],
       [ 32],
       [ 72],
       [138],
       [111],
       [ 48],
       [ 23],
       [ 19],
       [ 27],
       [ 59],
       [129],
       [129],
       [ 58],
       [ 27],
       [ 19],
       [ 24],
       [ 46],
       [112],
       [144],
       [ 73],
       [ 30],
       [ 20],
       [ 19],
       [ 37],
       [ 92],
       [152],
       [ 93],
       [ 36],
       [ 20],
       [ 18],
       [ 29],
       [ 71],
       [146],
       [117],
       [ 46],
       [ 23],
       [ 18],
       [ 22],
       [ 52],
       [128],
       [142],
       [ 62],
       [ 26],
       [ 17],
       [ 19],
       [ 37],
       [100],
       [158],
       [ 86],
       [ 32],
       [ 17],
       [ 17],
       [ 27],
       [ 72],
       [154],
       [118],
       [ 43],
       [ 20],
       [ 15],
       [ 21],
       [ 47],
       [128],
       [150],
       [ 63],
       [ 24],
       [ 16],
      